# Inspect Containers (Recursively)

In [1]:
# default_exp inspector

In [41]:
# export
from torch_snippets.loader import *
from torch_snippets.registry import AttrDict


def inspect(*arrays, **kwargs):
    """
    shows shape, min, max and mean of an array/list/dict of oreys
    Usage:
    inspect(arr1, arr2, arr3, [arr4,arr5,arr6], arr7, [arr8, arr9],...)
    where every `arr` is  assume to have a .shape, .min, .max and .mean methods
    """
    depth = kwargs.pop("depth", 0)
    names = kwargs.pop("names", None)

    if names is not None:
        if "," in names:
            names = names.split(",")
        assert len(names) == len(
            arrays
        ), "Give as many names as there are tensors to inspect"
    if depth == 0:
        line()
    for ix, arr in enumerate(arrays):
        name = "\t" * depth
        name = (
            name + f"{names[ix].upper().strip()}:\n" + name
            if names is not None
            else name
        )
        name = name
        typ = type(arr).__name__

        if isinstance(arr, AttrDict) and (AttrDict not in kwargs.get("suppress", [])):
            arr = arr.to_dict()
            inspect(arr, depth=depth + 1, **kwargs)

        elif isinstance(arr, (L, list, tuple)):
            print(f"{name}{typ} of {len(arr)} items")
            inspect(*arr[: kwargs.get("max_items", 5)], depth=depth + 1, **kwargs)
            if len(arr) > kwargs.get("max_items", 5):
                print(
                    "\t" * (depth + 1)
                    + f"and ... ... {len(arr) - kwargs.get('max_items', 5)} more item(s)"
                )

        elif isinstance(arr, dict) and (dict not in kwargs.get("suppress", [])):
            print(f"{name}{typ} of {len(arr)} items")
            for ix, (k, v) in enumerate(arr.items()):
                inspect(v, depth=depth + 1, names=[k])
                if ix == kwargs.get("max_items", 5) - 1:
                    break

            if len(arr) > kwargs.get("max_items", 5):
                print(
                    "\t" * (depth)
                    + f"... ... {len(arr) - kwargs.get('max_items', 5)} more item(s)"
                )

        elif isinstance(arr, pd.DataFrame):
            print(f"{name}{typ}\tShape: {arr.shape}")

        elif isinstance(arr, BB):
            info = f"{name}{typ}\t{arr}"

        elif hasattr(arr, "shape"):
            sh, m, M, dtype = arr.shape, arr.min(), arr.max(), arr.dtype
            try:
                me = arr.mean()
            except:
                me = arr.float().mean()
            info = f"{name}{typ}\tShape: {sh}\tMin: {m:.3f}\tMax: {M:.3f}\tMean: {me:.3f}\tdtype: {dtype}"
            if hasattr(arr, "device"):
                info += f" @ {arr.device}"
            print(info)

        elif isinstance(arr, str):
            if len(arr) > 50:
                arr = arr[:25] + "..." + arr[-25:]
            print(f"{name}{typ} `{arr}`")
        else:
            try:
                ln = len(arr)
                print(f"{name}{typ} Length: {ln}")
            except:
                print(f"{name}{typ}: {arr}")
        if depth == 0:
            line()

In [44]:
x = {
    "a": [0, 1, 2, 3],
    "b": torch.rand(10, 10),
    "c": {
        "d": np.arange(10),
        "e": [
            "np.arange",
            {},
            tuple(
                [
                    1,
                    2,
                ]
            ),
            set([1, 2, 3]),
            [],
            [11, 10],
        ],
    },
}
inspect(x)

dict of 3 items
	A:
	list of 4 items
		int: 0
		int: 1
		int: 2
		int: 3
	B:
	Tensor	Shape: torch.Size([10, 10])	Min: 0.008	Max: 0.999	Mean: 0.521	dtype: torch.float32
	C:
	dict of 2 items
		D:
		ndarray	Shape: (10,)	Min: 0.000	Max: 9.000	Mean: 4.500	dtype: int64
		E:
		list of 6 items
			str `np.arange`
			dict of 0 items
			tuple of 2 items
				int: 1
				int: 2
			set Length: 3
			list of 0 items
			and ... ... 1 more item(s)
